In [1]:
import pandas as pd

In [18]:
df = pd.read_csv("../xgbetting/processed/pl_to_20180924.csv", index_col=0, parse_dates=["Date"], dayfirst=True)

In [19]:
df.columns

Index(['ABP', 'AC', 'AF', 'AHW', 'AO', 'AR', 'AS', 'AST', 'AY', 'Attendance',
       'AwayTeam', 'B365<2.5', 'B365>2.5', 'B365A', 'B365D', 'B365H', 'BWA',
       'BWD', 'BWH', 'Bb1X2', 'BbAvH', 'BbMxD', 'BbMxH', 'Date', 'Div', 'FTAG',
       'FTHG', 'FTR', 'GB<2.5', 'GB>2.5', 'GBA', 'GBD', 'GBH', 'HBP', 'HC',
       'HF', 'HHW', 'HO', 'HR', 'HS', 'HST', 'HTAG', 'HTHG', 'HTR', 'HY',
       'HomeTeam', 'IWA', 'IWD', 'IWH', 'LBA', 'LBD', 'LBH', 'PSA', 'PSD',
       'PSH', 'Referee', 'SBA', 'SBD', 'SBH', 'SJA', 'SJD', 'SJH', 'SOA',
       'SOD', 'SOH', 'SYA', 'SYD', 'SYH', 'VCA', 'VCD', 'VCH', 'WHA', 'WHD',
       'WHH', 'season'],
      dtype='object')

In [20]:
df = df[["HomeTeam", "AwayTeam", "Date", "season", "FTHG", "FTAG", "FTR", "BbMxD", "BbMxH"]]

In [21]:
df.sort_values("Date")

,HomeTeam,AwayTeam,Date,season,FTHG,FTAG,FTR,BbMxD,BbMxH
0,Charlton,Everton,2001-08-18,102,1.0,2.0,A,NaN,NaN
1,Derby,Blackburn,2001-08-18,102,2.0,1.0,H,NaN,NaN
2,Leeds,Southampton,2001-08-18,102,2.0,0.0,H,NaN,NaN
3,Leicester,Bolton,2001-08-18,102,0.0,5.0,A,NaN,NaN
4,Liverpool,West Ham,2001-08-18,102,2.0,1.0,H,NaN,NaN
5,Middlesbrough,Arsenal,2001-08-18,102,0.0,4.0,A,NaN,NaN
6,Sunderland,Ipswich,2001-08-18,102,1.0,0.0,H,NaN,NaN
7,Tottenham,Aston Villa,2001-08-18,102,0.0,0.0,D,NaN,NaN
9,Man United,Fulham,2001-08-19,102,3.0,2.0,H,NaN,NaN
8,Chelsea,Newcastle,2001-08-19,102,1.0,1.0,D,NaN,NaN
